This notebooks is not a one off.

I keep coming back to it to make different version of the dataset.

In [14]:
import sys
sys.path.append("../scripts")
from fetch_data import DataLoader
from cleaning import CleanDataFrame
from visualization import Plotters
from processing import FeatureEngineering
from exploration import Analysis

In [15]:
cleaner = CleanDataFrame()
plotters = Plotters(w=8, h=8)
analyzer = Analysis()


In [20]:
DATASET_PATH = "../data/Debernardi et al 2020 data.csv"

In [21]:
df = DataLoader.read_csv(DATASET_PATH)

Pandas: CSV read from: ../data/Debernardi et al 2020 data.csv


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sample_id                590 non-null    object 
 1   patient_cohort           590 non-null    object 
 2   sample_origin            590 non-null    object 
 3   age                      590 non-null    int64  
 4   sex                      590 non-null    object 
 5   diagnosis                590 non-null    int64  
 6   stage                    199 non-null    object 
 7   benign_sample_diagnosis  208 non-null    object 
 8   plasma_CA19_9            350 non-null    float64
 9   creatinine               590 non-null    float64
 10  LYVE1                    590 non-null    float64
 11  REG1B                    590 non-null    float64
 12  TFF1                     590 non-null    float64
 13  REG1A                    306 non-null    float64
dtypes: float64(6), int64(2), o

For this first version, I will remove all the patient related attributes that don't relate to the disease and don't contribute to the generality of my model.
These will be columns like the `sample_id` which is unique for each patient.

- sample_id
- patient_cohort
- sample_origin
- stage
- benign_sample_diagnosis

In [23]:
col_to_drop = [
    "sample_id",
    "patient_cohort",
    "sample_origin",
    "stage",
    "benign_sample_diagnosis",
]
df = cleaner.drop_columns(df, col_to_drop)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            590 non-null    int64  
 1   sex            590 non-null    object 
 2   diagnosis      590 non-null    int64  
 3   plasma_CA19_9  350 non-null    float64
 4   creatinine     590 non-null    float64
 5   LYVE1          590 non-null    float64
 6   REG1B          590 non-null    float64
 7   TFF1           590 non-null    float64
 8   REG1A          306 non-null    float64
dtypes: float64(6), int64(2), object(1)
memory usage: 41.6+ KB


In [27]:
count, cols = analyzer.get_missing_entries_count(df)

In [31]:
count

plasma_CA19_9    240
REG1A            284
dtype: int64

In [25]:
analyzer.percent_missing(df)

The dataset contains 9.87 % missing values.


# Save the data

In [26]:
df.to_csv(DATASET_PATH, index=False)

# Deal with missing values

In [28]:
df = cleaner.replace_missing(df, columns=cols, method="median")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            590 non-null    int64  
 1   sex            590 non-null    object 
 2   diagnosis      590 non-null    int64  
 3   plasma_CA19_9  590 non-null    float64
 4   creatinine     590 non-null    float64
 5   LYVE1          590 non-null    float64
 6   REG1B          590 non-null    float64
 7   TFF1           590 non-null    float64
 8   REG1A          590 non-null    float64
dtypes: float64(6), int64(2), object(1)
memory usage: 41.6+ KB


In [29]:
analyzer.get_univariate_analysis(df)

,count,mean,std,min,25%,50%,75%,max,mode,CoV,skew,kurtosis,iqr,missing_counts
age,590.0,59.079661,13.109520,26.000000,50.000000,60.000000,69.000000,89.000000,68.000000,0.221896,-0.216832,-0.511899,19.000000,0
diagnosis,590.0,2.027119,0.804873,1.000000,1.000000,2.000000,3.000000,3.000000,2.000000,0.397053,-0.049186,-1.454333,2.000000,0
plasma_CA19_9,590.0,398.747509,1896.028213,0.000000,17.000000,26.500000,41.750000,31000.000000,26.500000,4.754959,10.376093,136.966308,24.750000,0
creatinine,590.0,0.855383,0.639028,0.056550,0.373230,0.723840,1.139482,4.116840,0.576810,0.747067,1.466413,2.739650,0.766253,0
LYVE1,590.0,3.063530,3.438796,0.000129,0.167179,1.649862,5.205037,23.890323,0.000814,1.122495,1.394014,2.419378,5.037858,0
REG1B,590.0,111.774090,196.267110,0.001104,10.757216,34.303353,122.741013,1403.897600,1.769536,1.755927,3.333925,13.077588,111.983796,0
TFF1,590.0,597.868722,1010.477245,0.005293,43.961000,259.873974,742.736000,13344.300000,0.024701,1.690132,5.158302,47.635879,698.775000,0
REG1A,590.0,394.105176,1120.564240,0.000000,26.500000,26.500000,224.007000,13200.000000,26.500000,2.843313,6.107913,48.241487,197.507000,0


# Save again

In [30]:
df.to_csv(DATASET_PATH, index=False)